In [3]:
import torch
import torchvision.transforms as transforms
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import pdb
import torch.optim as optim
# params
crop_size = 400

# get image
img = imageio.imread('ca.jpg')[...,:3]/255.
# plt.imshow(img)
# plt.show()

# convert to tensor
img_tensor = torch.Tensor(img).permute(2, 0, 1)
# print(img_tensor)
img_cropped = transforms.CenterCrop(crop_size)(img_tensor)
img_flatten = torch.reshape(img_cropped, (crop_size * crop_size, 3))


# test
# img_crop = img_cropped.permute(1, 2, 0)
# img_crop = img_flatten.view( crop_size, crop_size, 3)


# Now, img_reshaped should have the same shape and values as img_cropped.


# img_crop = torch.reshape(img_cropped, (crop_size, crop_size, 3))
# img_crop =torch.reshape(img_flatten, (crop_size, crop_size, 3))
# img_crop = img_flatten.unsqueeze(0)
# img_crop = img_crop.reshape(img_crop, (crop_size, crop_size, 3))
# img_crop.detach().numpy()
# print(img_crop.shape)


# plt.imshow(img_crop)
# plt.show()

# create the mesh grid
xy_range = list(map(lambda x: (x / (crop_size - 1) * 2) - 1, range(crop_size)))
xy_range_tensor = torch.Tensor(xy_range)
x_grid, y_grid = torch.meshgrid(xy_range_tensor, xy_range_tensor, indexing='ij')
xy_coord_tensor = torch.stack((x_grid, y_grid), dim= -1)
print(f'xy_coord_tensor:{xy_coord_tensor.shape}')
xy_flatten = torch.reshape(xy_coord_tensor, (crop_size * crop_size, 2))



# pdb.set_trace()


import torch.nn as nn


# mlp
class MLP(torch.nn.Module):
    def __init__(
        self,
        in_features: int,
        hidden_features: int,
        hidden_layers: int,
        out_features: int
    ):
        super().__init__()
        out_features = out_features if out_features is not None else hidden_features
        layers = [
            torch.nn.Linear(in_features, hidden_features),
            torch.nn.BatchNorm1d(hidden_features),  # Batch Normalization layer
            torch.nn.ReLU(),
        ]
        for _ in range(hidden_layers):
            layers.extend([
                torch.nn.Linear(hidden_features, hidden_features),
                torch.nn.BatchNorm1d(hidden_features),  # Batch Normalization layer
                torch.nn.ReLU(),
            ])
        layers.append(torch.nn.Linear(hidden_features, out_features))
        layers.append(torch.nn.Sigmoid())  # Assuming you want to keep the Sigmoid on the output layer

        self.net = torch.nn.Sequential(*layers)
    
    def forward(self, x: torch.Tensor):
        return self.net(x)

# declare model
net = MLP(in_features=2, hidden_features=32, hidden_layers=8, out_features=3)

model = net

# print(model)

# params
learning_rate = 0.01
num_epochs = 701

# set the target
# target = img_flatten
target = img_cropped

# optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

for epoch in range(num_epochs):

    model.train()

    # for mlp
    generated = model(xy_coord_tensor)
    generated = generated.permute(2, 0, 1)
    print(f'generated_shape: {generated.shape}')

    # loss = criterion(generated, target)
    loss = nn.functional.l1_loss(target, generated)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch %d, loss = %.03f' % (epoch, float(loss)))
        # print(generated.shape)
        # generated = generated.view((crop_size, crop_size, 3)).detach().numpy()
        print(generated.shape)
        # generated = torch.reshape(generated, (crop_size, crop_size, 3)).detach().numpy()
        # generated = generated * 255
        print(f'generated: {generated}')
        # generated = (generated * 255).astype(np.uint8)
        # generated = (generated - generated.min()) / (generated.max() - generated.min())
        # print(generated)
        generated = generated.permute(1, 2, 0).detach().numpy()
        plt.imshow(generated)
        # plt.show()
        plt.pause(10)
        plt.close()
model.eval()


xy_coord_tensor:torch.Size([400, 400, 2])


RuntimeError: running_mean should contain 400 elements not 32

In [1]:
from img_dataset import xy_coord_tensor



xy_coord_tensor = xy_coord_tensor.unsqueeze(0).permute(0, 3, 1, 2)

import torch.nn as nn


class CNN(nn.Module):
    def __init__(self, in_features, hidden_features, out_features):
        super().__init__()

        self.neural_cnn_net = nn.Sequential(
            
            nn.Conv2d(
                in_features,
                hidden_features,
                kernel_size=1,
                padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(hidden_features),

            nn.Conv2d(
                hidden_features,
                hidden_features,
                kernel_size=1,
                padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(hidden_features),

            nn.Conv2d(
                hidden_features,
                hidden_features,
                kernel_size=1,
                padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(hidden_features),

            nn.Conv2d(
                hidden_features,
                out_features,
                kernel_size=1,
                padding=0),
            nn.Sigmoid(),

        )

    def forward(self, x):
        return self.neural_cnn_net(x)

# declare model
net = CNN(in_features=2, hidden_features=300, out_features=3)

model = net

from img_dataset import img_cropped, xy_coord_tensor
from cnn_model import model
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt
import torch
from img_dataset import img_cropped
from img_dataset_cnn import xy_coord_tensor

# params
learning_rate = 0.01
num_epochs = 701

# set the target
# target = img_flatten
img_cropped = img_cropped.unsqueeze(0)

target = img_cropped

# optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

for epoch in range(num_epochs):

    model.train()

    # for mlp
    generated = model(xy_coord_tensor)
    # generated = generated.permute(2, 0, 1)
    # print(f'generated_shape: {generated.shape}')

    # loss = criterion(generated, target)
    loss = nn.functional.l1_loss(target, generated)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch %d, loss = %.03f' % (epoch, float(loss)))
        # print(generated.shape)
        # generated = generated.view((crop_size, crop_size, 3)).detach().numpy()
        # print(generated.shape)
        # generated = torch.reshape(generated, (crop_size, crop_size, 3)).detach().numpy()
        # generated = generated * 255
        print(f'generated: {generated}')
        # generated = (generated * 255).astype(np.uint8)
        # generated = (generated - generated.min()) / (generated.max() - generated.min())
        # print(generated)
        generated = generated.squeeze(0).permute(1, 2, 0).detach().numpy()
        plt.imshow(generated)
        plt.show()
        # plt.pause(10)
        # plt.close()
model.eval()


img shape: (412, 550, 3)


RuntimeError: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 2 is not equal to len(dims) = 3